## 1. 자카드 유사도

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
#tfidf로 탑 10 뽑기
from math import log10
from konlpy.tag import Okt

# =======================================
# -- TF-IDF function
# =======================================
def f(t, d):
    # d is document == tokens
    return d.count(t)

def tf(t, d):
    # d is document == tokens
    return 0.5 + 0.5*f(t,d)/max([f(w,d) for w in d])

def idf(t, D):
    # D is documents == document list
    numerator = len(D)
    denominator = 1 + len([ True for d in D if t in d])
    return log10(numerator/denominator)

def tfidf(t, d, D):
    return tf(t,d)*idf(t, D)

def tokenizer(d):
    okt = Okt()
    def keyword_extractor(text):
        tokens = okt.phrases(text)
        tokens = [token for token in tokens if len(token) > 1]  # 한 글자인 단어는 제외
        count_dict = [(token, text.count(token)) for token in tokens]
        ranked_words = sorted(count_dict, key=lambda x: x[1], reverse=True)[:20]
        return [keyword for keyword, freq in ranked_words]
    return keyword_extractor(d)

def tfidfScorer(D):
    tokenized_D = [tokenizer(d) for d in D]
    result = []
    for d in tokenized_D:
        result.append([t for t in d])
    return result

In [ ]:
#탑 10뽑아내기  #해당 명사들이 jaccard 유사도의 input으로 추가됨
tfidf = list()
for id, s in enumerate( tfidfScorer(df['answer']) ):
    s = sorted(s, key=lambda x:x[1], reverse=True)
    tfidf.append(s[:10])

In [ ]:
def jaccard(sen1, sen2):
    a = set(sen1) 
    b = set(sen2) 
    aUb = a.union(b) 
    aNb = a.intersection(b) 
    p = len(aNb) / len(aUb) 
    return p

## 2. Document similarity(w2v을 이용한 문장 간 유사도)

In [ ]:
# word2vec 임베딩 모델 로드
from sklearn.externals import joblib
embedding_model = joblib.load("./skipgram_ver1_1.sav")

In [ ]:
def docsim(a,b):
    index2word_set = set(embedding_model.wv.index2word)  

    def avg_feature_vector(sentence, model, num_features, index2word_set):  #w2v을 이용한 평균 임베딩 벡터 도출 함수
        words = str(sentence).split(" ")
        feature_vec = np.zeros((num_features, ), dtype='float32')
        n_words = 0
        for word in words:
            if word in index2word_set:
                n_words += 1
                feature_vec = np.add(feature_vec, model[word])
        if (n_words > 0):
            feature_vec = np.divide(feature_vec, n_words)
        return feature_vec
    s1_afv = avg_feature_vector(a, model=embedding_model, num_features=100, index2word_set=index2word_set)  
    s2_afv = avg_feature_vector(b, model=embedding_model, num_features=100, index2word_set=index2word_set)
    sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)  #코사인 유사도 계산
    return sim

## 3. TF-IDF 행렬을 이용한 코사인 유사도

In [ ]:
def get_recom(a,b):    
    data = pd.DataFrame(data=[a,b], index=range(0,2),columns = ["o"])
    tfidf_matrix = tfidf.fit_transform(data["o"])
    
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    final = cosine_sim[0][1]
    return final